# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import glob
import os 

RAWDATA = "./rawdata"
PROCESSEDDATA = "./processeddata"
STEPWIDTH = 3600

In [2]:
# Read rawdata
data_frames = []
data_names = []
for raw_file in glob.glob(RAWDATA + "/*.csv"):
    data_frames.append(pd.read_csv(os.path.join(raw_file)))
    name = os.path.basename(raw_file)
    name = name.split("-")[0]
    data_names.append(name)
print("Found coin data: " + str(data_names))

Found coin data: ['XRPBTC', 'ETHBTC', 'LTCBTC', 'BTCUSDT', 'ADABTC', 'XMRBTC']


In [3]:
#print(data_frames[0].columns)
# Available: 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'

# Drop unused data
for idx, _ in enumerate(data_frames):
    data_frames[idx].drop(columns=['volume','close_time','quote_av','trades','tb_base_av','tb_quote_av','ignore'], inplace= True)
print("Available columns: " + str(data_frames[0].columns))

# Convert timestamp 
for idx, _ in enumerate(data_frames):
    data_frames[idx]['timestamp'] = pd.to_datetime(data_frames[idx]['timestamp'])
    data_frames[idx]['unixtime'] = (data_frames[idx]['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
print("Available columns: " + str(data_frames[0].columns))

Available columns: Index(['timestamp', 'open', 'high', 'low', 'close'], dtype='object')
Available columns: Index(['timestamp', 'open', 'high', 'low', 'close', 'unixtime'], dtype='object')


In [4]:
# Find latest beginning timestamp 
latest_begin = 0
latest_coin_idx = -1
for idx, _ in enumerate(data_frames):
    if (data_frames[idx]['unixtime'][0] > latest_begin):
        latest_coin_idx = idx 
        latest_begin = data_frames[idx]['unixtime'][0]
print("Coin with latest data start: " + data_names[latest_coin_idx] + " " + str(data_frames[latest_coin_idx]['timestamp'][0]))

# Drop data points earlier than latest data start 
for idx, _ in enumerate(data_frames):
    data_frames[idx].drop(data_frames[idx][data_frames[idx]['unixtime'] < latest_begin].index, inplace=True)
    data_frames[idx].reset_index(inplace=True)
    data_frames[idx].drop(columns=["index"], inplace=True)
    print(data_names[idx] + " length: " + str(len(data_frames[idx])) + " ", end='')
print("\nDropped all coin's previous data.")

Coin with latest data start: ADABTC 2017-11-30 12:00:00
XRPBTC length: 29577 ETHBTC length: 29587 LTCBTC length: 29587 BTCUSDT length: 29577 ADABTC length: 29577 XMRBTC length: 29587 
Dropped all coin's previous data.


In [5]:
# Fill holes in the data 
for idx, _ in enumerate(data_frames):
    start_time = data_frames[idx]['unixtime'][0]
    end_time = data_frames[idx]['unixtime'].iloc[-1]
    
    # Find holes from unixtime and create missing lines filled with nans
    new_index = pd.Index(np.arange(start_time,end_time,STEPWIDTH), name="unixtime")
    data_frames[idx] = data_frames[idx].set_index('unixtime').reindex(new_index).reset_index()
    
    # Keep track of where data was added
    data_frames[idx]['interpolateddata'] = data_frames[idx]['timestamp'].isnull()
    
    # Calculate date time timestamp from unixtime for added lines
    data_frames[idx].loc[data_frames[idx]['interpolateddata'] == True, 'timestamp'] = data_frames[idx].loc[data_frames[idx]['interpolateddata'] == True, 'unixtime'].apply(lambda x: pd.to_datetime(x, unit='s'))
    
    # Fill line's empty columns 
    data_frames[idx]['open'].fillna(method='ffill', inplace = True)
    data_frames[idx]['high'].fillna(method='ffill', inplace = True)
    data_frames[idx]['low'].fillna(method='ffill', inplace = True)
    data_frames[idx]['close'].fillna(method='ffill', inplace = True)
    data_frames[idx]['open'].fillna(method='bfill', inplace = True)
    data_frames[idx]['high'].fillna(method='bfill', inplace = True)
    data_frames[idx]['low'].fillna(method='bfill', inplace = True)
    data_frames[idx]['close'].fillna(method='bfill', inplace = True)
    
    print(data_names[idx] + " length: " + str(len(data_frames[idx])) + " Found NaNs: " + str(data_frames[idx].isnull().values.any()))

XRPBTC length: 29686 Found NaNs: False
ETHBTC length: 29686 Found NaNs: False
LTCBTC length: 29686 Found NaNs: False
BTCUSDT length: 29686 Found NaNs: False
ADABTC length: 29686 Found NaNs: False
XMRBTC length: 29686 Found NaNs: False


In [6]:
# calculate prices based on US Dollar (use USDT-BTC prices, as USDT is stable coin, i.e. always roughly 1 Dollar)
usdt_btc_idx = data_names.index('BTCUSDT')
for idx, _ in enumerate(data_frames):
    if idx == usdt_btc_idx:
        continue
    # rename original prices 
    data_frames[idx].rename(columns={'open':'openbtc','close':'closebtc','high':'highbtc','low':'lowbtc'},inplace = True)
    data_frames[idx]['open'] = data_frames[idx]['openbtc'] * data_frames[usdt_btc_idx]['open']
    data_frames[idx]['close'] = data_frames[idx]['closebtc'] * data_frames[usdt_btc_idx]['close']
    data_frames[idx]['low'] = data_frames[idx]['lowbtc'] * data_frames[usdt_btc_idx]['low']
    data_frames[idx]['high'] = data_frames[idx]['highbtc'] * data_frames[usdt_btc_idx]['high']

In [7]:
# Calculate special indices 


# Average Directional Index (ADX) (credit to https://stackoverflow.com/a/64946213)
def ADX(data: pd.DataFrame, period: int):
    """
    Computes the ADX indicator.
    """
    
    df = data.copy()
    alpha = 1/period

    # TR
    df['H-L'] = df['high'] - df['low']
    df['H-C'] = np.abs(df['high'] - df['close'].shift(1))
    df['L-C'] = np.abs(df['low'] - df['close'].shift(1))
    df['TR'] = df[['H-L', 'H-C', 'L-C']].max(axis=1)
    del df['H-L'], df['H-C'], df['L-C']

    # ATR
    df['ATR'] = df['TR'].ewm(alpha=alpha, adjust=False).mean()

    # +-DX
    df['H-pH'] = df['high'] - df['high'].shift(1)
    df['pL-L'] = df['low'].shift(1) - df['low']
    df['+DX'] = np.where(
        (df['H-pH'] > df['pL-L']) & (df['H-pH']>0),
        df['H-pH'],
        0.0
    )
    df['-DX'] = np.where(
        (df['H-pH'] < df['pL-L']) & (df['pL-L']>0),
        df['pL-L'],
        0.0
    )
    del df['H-pH'], df['pL-L']

    # +- DMI
    df['S+DM'] = df['+DX'].ewm(alpha=alpha, adjust=False).mean()
    df['S-DM'] = df['-DX'].ewm(alpha=alpha, adjust=False).mean()
    df['+DMI'] = (df['S+DM']/df['ATR'])*100
    df['-DMI'] = (df['S-DM']/df['ATR'])*100
    del df['S+DM'], df['S-DM']

    # ADX
    df['DX'] = (np.abs(df['+DMI'] - df['-DMI'])/(df['+DMI'] + df['-DMI']))*100
    df['ADX'] = df['DX'].ewm(alpha=alpha, adjust=False).mean()
    del df['DX'], df['ATR'], df['TR'], df['-DX'], df['+DX'], df['+DMI'], df['-DMI']

    return df


for idx, _ in enumerate(data_frames):
    
    # Moving average convergence divergence (MACD) (credit to https://towardsdatascience.com/implementing-macd-in-python-cc9b2280126a)
    exp1 = data_frames[idx]['close'].ewm(span=12, adjust=False).mean()
    exp2 = data_frames[idx]['close'].ewm(span=26, adjust=False).mean()
    data_frames[idx]['macd'] = exp1-exp2
    print("Finished MACD for: " + data_names[idx])
    
    # Relative Strength Index (RSI) (credit to https://stackoverflow.com/a/29400434)
    delta = data_frames[idx]['close'].diff()
    delta = delta[1:] # first row is nan, get rid of it 
    up, down = delta.copy(), delta.copy() # Make the positive gains (up) and negative gains (down) Series
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.ewm(span=12).mean() # Calculate the EWMA
    roll_down = down.abs().ewm(span=12).mean()
    RS = roll_up / roll_down # Calculate the RSI based on EWMA
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data_frames[idx]['rsi'] = pd.concat([pd.Series([np.nan]), RSI])
    print("Finished RSI for: " + data_names[idx])

    # Commodity Channel Index (CCI) (credit to https://blog.quantinsti.com/build-technical-indicators-in-python/#commodity-channel-index) 
    TP = (data_frames[idx]['high'] + data_frames[idx]['low'] + data_frames[idx]['close']) / 3 
    SMA = TP.rolling(14).mean()
    MAD = TP.rolling(14).apply(lambda x: pd.Series(x).mad())
    data_frames[idx]['cci'] = (TP - SMA) / (0.015 * MAD) 
    print("Finished CCI for: " + data_names[idx])
    
    # Average Directional Index (ADX) (credit to https://stackoverflow.com/a/64946213)
    data_frames[idx]['adx'] = ADX(data_frames[idx],12)['ADX']
    print("Finished ADX for: " + data_names[idx])
    
    # Delete first few data points to account for incorrect moving average starting values
    data_frames[idx].drop(data_frames[idx].index[:15], inplace=True)
    data_frames[idx].reset_index(inplace=True)
    data_frames[idx].drop(columns=["index"], inplace=True)




Finished MACD for: XRPBTC
Finished RSI for: XRPBTC
Finished CCI for: XRPBTC
Finished ADX for: XRPBTC
Finished MACD for: ETHBTC
Finished RSI for: ETHBTC
Finished CCI for: ETHBTC
Finished ADX for: ETHBTC
Finished MACD for: LTCBTC
Finished RSI for: LTCBTC
Finished CCI for: LTCBTC
Finished ADX for: LTCBTC
Finished MACD for: BTCUSDT
Finished RSI for: BTCUSDT
Finished CCI for: BTCUSDT
Finished ADX for: BTCUSDT
Finished MACD for: ADABTC
Finished RSI for: ADABTC
Finished CCI for: ADABTC
Finished ADX for: ADABTC
Finished MACD for: XMRBTC
Finished RSI for: XMRBTC
Finished CCI for: XMRBTC
Finished ADX for: XMRBTC


In [8]:
# Final available data
print("Available columns: " + str(data_frames[0].columns))

# Write to disk 
for idx, _ in enumerate(data_frames):
    data_frames[idx].to_csv(os.path.join(PROCESSEDDATA, data_names[idx] + ".csv"))
print("Wrote data to disk.")

Available columns: Index(['unixtime', 'timestamp', 'openbtc', 'highbtc', 'lowbtc', 'closebtc',
       'interpolateddata', 'open', 'close', 'low', 'high', 'macd', 'rsi',
       'cci', 'adx'],
      dtype='object')
Wrote data to disk.
